# Lab-3 Qubit Allocation and Routing Algorithms

## Instruction

### For each question, you need to complete the associated 'question*.py' file by implementing the functions for that question.

### How to Submit

#### 1. Submit question1.py, question2.py, and question3.py to Gradescope Lab-3.

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, transpiler
from qiskit.visualization import circuit_drawer
from qiskit_aer.backends.qasm_simulator import QasmSimulator
from qiskit.circuit import Gate, ControlledGate
from qiskit.circuit.library import CUGate
from qiskit.quantum_info import Statevector
import math
import matplotlib
import numpy as np
import glob
import os
import statistics

In [3]:
### Utils
def get_benchmark_dict(path):
    """
    Finds and reads benchmark quantum circuits from *.qasm files in the given
    path and store them in a dict using *.qasm filenames as keys and
    the QuantumCircuit objects as values.
    Args:
        path: str
    Return:
        out_dict: dict
    """
    list_qasm = glob.glob(os.path.join(path,'*.qasm'))
    out_dict = {}
    for qasm_file in list_qasm:
        circuit_name = qasm_file.split('.qasm')[0].split('Benchmarks'+os.sep)[1]
        circuit = QuantumCircuit.from_qasm_file(qasm_file)
        circuit.name = circuit_name
        out_dict.update({circuit_name:circuit})
    
    return out_dict
workload_list = get_benchmark_dict("SWAP_Benchmarks")

## Q1. Decomposition of Benchmark Circuits [20 Points]

### In the Lab-3/SWAP-Benchmarks, we have seven quantum benchmark circuits in ".qasm" format. Unfortunately, not all of them use the basis gate set that IBM uses for physical execution. This problem can be solved by picking a basis set of gates, and every gate in the benchmark circuit can be expressed as a composition of gates in the basis set, this process is called the decomposing the circuit.

### Q1A. [10 Points] In `q1a` function in `question1.py`, decompose the benchmark circuits using qiskit transpile function with basis gates set `[cx,u]`, and return a dictionary of benchmark circuits' names as keys and transpiled circuits as values.
##### You are expected to use the same names of circuits generated by `get_benchmark_dict`.

In [4]:
q1a_basis_gates = ['cx', 'u']
output_dict_1a = {}
for benchmark_circuit in workload_list.keys():
    circuit = workload_list[benchmark_circuit]
    transpiled_1a = transpile(circuit, basis_gates=q1a_basis_gates)
    output_dict_1a[benchmark_circuit] = transpiled_1a

### Q1B. [10 Points] In `q1b` function in `question1.py`, decompose the benchmark circuits using qiskit transpile function with basis gates set `[cx,rx,ry,rz]`, and return a dictionary of benchmark circuits' names as keys and transpiled circuits as values.
##### You are expected to use the same names of circuits generated by `get_benchmark_dict`.

In [5]:
q1b_basis_gates = ['cx', 'rx', 'ry', 'rz']
output_dict_1b = {}
for benchmark_circuit in workload_list.keys():
    circuit = workload_list[benchmark_circuit]
    transpiled = transpile(circuit, basis_gates=q1b_basis_gates)
    output_dict_1b[benchmark_circuit] = transpiled

## Q2. Applying SABRE Algorithm for Device Topology [30 Points]

### In this question, we learn how SABRE algorithm can be used to map quantum circuits to given quantum device topology. We will use qiskit transpile function to apply SABRE algorithm, and evaluate the change of circuit depth and the number of SWAPs inserted for each benchmark circuits averaged over all coupling maps:

##### `"Grid 5X5"` <img src="./figures/q2_device_grid_5x5.png" alt="NISQ" height=200/>
##### `"Grid 5X4"` <img src="./figures/q2_device_grid_5x4.png" alt="NISQ" height=200/>
##### `"Grid 7X3"` <img src="./figures/q2_device_grid_7x3.png" alt="NISQ" height=200/>
##### `"Ring 20"` <img src="./figures/q2_device_ring_20.png" alt="NISQ" height=200/>



### Q2A. [10 Points] In `create_coupling_maps` function in `question2.py`, generate each coupling map as shown above using transpiler.CouplingMap and store them in the dictionary provided using the given keys.

In [6]:
coupling_maps = {}
coupling_maps['GRID 5X5'] = None
coupling_maps['GRID 5X4'] = None
coupling_maps['GRID 7X3'] = None
coupling_maps['RING 20'] = None

for grid in [(5, 5, 'GRID 5X5'), (5, 4, 'GRID 5X4'), (7, 3, 'GRID 7X3')]:
    height = grid[0]
    width = grid[1]
    grid_edges = []
    # starting from maximum qubit number
    # connect horizontal edges
    for row in range(height):
        for col in range(width - 1):
            qubit = row * width + col
            grid_edges.append([qubit, qubit + 1])
            grid_edges.append([qubit + 1, qubit])
    # connect vertical edges
    for col in range(width):
        for row in range(height - 1):
            qubit = row * width + col
            # essentially qubit one row above via adding width
            grid_edges.append([qubit, (row + 1) * width + col])
            grid_edges.append([(row + 1) * width + col, qubit])
    c_map = transpiler.CouplingMap(couplinglist=grid_edges)
    coupling_maps[grid[2]] = c_map

In [7]:
ring_edges = []
for i in range(20):
    neighbor_forward = (i + 1) % 20
    # negative modulo: n - a mod n
    neighbor_backward = (i - 1) % 20
    ring_edges.append([i, neighbor_forward])
    ring_edges.append([i, neighbor_backward])
# remove duplicates in coupling map
c_map = transpiler.CouplingMap(couplinglist=ring_edges)
coupling_maps["RING 20"] = c_map

### Q2B. [10 Points] In `average_depth_change` function in `question2.py`, calculate the average change of circuit depth when transpiling the benchmark circuits for given coupling maps, using SABRE for routing and mapping. You need to return a dictionary with benchmark circuits' names as keys and change of circuit depth averaged over all coupling maps.
##### You are expected to use the same names of circuits generated by `get_benchmark_dict`.

In [8]:
output_dict = {}
for circuit_key, benchmark_circuit in workload_list.items():
    benchmark_circuit_depth = benchmark_circuit.depth()
    depth_diffs = []
    for _, c_map in coupling_maps.items():
        transpiled_benchmark_circuit = transpile(benchmark_circuit, coupling_map=c_map, routing_method='sabre', optimization_level=1)
        transpiled_depth = transpiled_benchmark_circuit.depth()
        diff = abs(benchmark_circuit_depth - transpiled_depth)
        depth_diffs.append(diff)
    mean_diff = statistics.mean(depth_diffs)
    output_dict[circuit_key] = mean_diff
print(output_dict)

{'BV-10': 1.5, 'BV-15': 7.5, 'qaoa10_depth2': 3, 'QFT-6': 2, 'QV9': 0}


### Q2C. [10 Points] In `average_nswap_change` function in `question2.py`, calculate the average number of swaps inserted when transpiling the benchmark circuits for given coupling maps, using SABRE for routing and mapping. You need to return a dictionary with benchmark circuits' names as keys and number of swaps inserted averaged over all coupling maps.
##### You are expected to use the same names of circuits generated by `get_benchmark_dict`.

In [9]:
swap_dict = {}
for circuit_key, benchmark_circuit in workload_list.items():
        swap_inserts = []
        # get swap gates before transpiling circuit, if any (default to 0)
        swap_count_before = benchmark_circuit.count_ops().get('swap', 0)
        for _, c_map in coupling_maps.items():
            transpiled_benchmark_circuit = transpile(benchmark_circuit, coupling_map=c_map, routing_method='sabre', optimization_level=1)
            swap_count_after = transpiled_benchmark_circuit.count_ops().get('swap', 0)
            diff = abs(swap_count_before - swap_count_after)
            swap_inserts.append(diff)
        average_swaps = statistics.mean(swap_inserts)
        swap_dict[circuit_key] = average_swaps
print(swap_dict)

{'BV-10': 1.5, 'BV-15': 7.75, 'qaoa10_depth2': 0, 'QFT-6': 2.5, 'QV9': 0}


## Q3. Simulation of a Noisy Device [50 Points]

### As shown in our lectures, real quantum machines suffer from noisy gates. From Lab-2 we learned that we can simulate noisy gates in qiskit using U gates. In this question, you need to first implement both noise-free and noisy CNOT gates, then practice qubit allocation by simulating a NISQ machine using your implementation of noisy CNOT gates.

### For simplicity, we define a noisy CNOT gate to act in the following way:
### A noisy CNOT gate with fidelity $p$, is a 2-qubit quantum gate, when the control qubit is at state 0, applies identity operation on the target qubit, when the control qubit is at state 1, applies NOT operation on the target with probability $p$ and applies identity operation on the target with probability $1-p$.

### Q3A. [5 points] Implementation of a noise-free CNOT gate

#### In `cx_ideal` function in `question3.py`, implement a noise-free CNOT gate without using `qiskit.circuit.library.CXGate`. You must return a `qiskit.circuit.Gate` object.

In [10]:
# rotate pi around Y, no phase shift, pi around Z, no global phase change
studentGate = CUGate(np.pi, 0, np.pi, 0)

### Q3B. [5 points] Implementation of noisy CNOT gates

#### In `cx_95` function in `question3.py`, implement a noisy CNOT gate as defined above with fidelity $p = 0.95$, You must return a `qiskit.circuit.Gate` object. Your implementation must follow the definition described in this question.

#### In `cx_70` function in `question3.py`, implement a noisy CNOT gate as defined above with fidelity $p = 0.70$, You must return a `qiskit.circuit.Gate` object. Your implementation must follow the definition described in this question.

In [11]:
p = 0.70
# p = 0.95
theta = 2 * math.asin(math.sqrt(p))
studentGate = CUGate(theta, 0,0, 0)

### Q3C. [20 Points] The figure below demonstrates a 4-qubit NISQ machine coupling map similar to the one we have seen in our lecture, where the value of each edge denotes the fidelity of a CNOT gate (in both directions) as defined in this question. We would like to run the following logical quantum circuit on this 4-qubit NISQ machine.

<img src="./figures/q3c_device_topology.png" alt="Q3C Device" height=200/>
<img src="./figures/q3_logical_circuit.png" alt="Target Quantum Circuit" height=200/>

#### In `q3c_edge_gates` function in `question3.py`, create noisy CNOT gates defined in this question based on the device topology.
#### In `q3c_device_compatible_physical_circuit` function in `question3.py`, implement the mapped physical quantum circuit on the coupling map. You need to consider the optimal way to allocate the qubits to maximize the overall fidelity. You can only use 2-qubit noisy CNOT gates in `q3c_edge_gates`.
#### In `q3c_device_qubit_route_mapping` function in `question3.py`, provide your logical qubit to physical qubit mapping in a dictionary in the following format `{logical_qubit: physical_qubit}`, this is the dictionary you use to allocate logical qubits in the circuit to the physical qubits in the device.
#### In `q3c_device_qubit_read_mapping` function in `question3.py`, provide your physical qubit to logical qubit mapping in a dictionary in the following format `{physical_qubit: logical_qubit}`, this is the dictionary you use to read from the physical device post execution and retrive the result for the logical quantum circuit.

In [12]:
edge_gates = {
        "01": None,
        "12": None,
        "23": None,
        "03": None,
    }

edge_gates["01"] = CUGate(2 * math.asin(math.sqrt(0.99)), 0, 0, 0)
edge_gates["12"] = CUGate(2 * math.asin(math.sqrt(0.90)), 0, 0, 0)
edge_gates["23"] = CUGate(2 * math.asin(math.sqrt(0.80)), 0, 0, 0)
edge_gates["03"] = CUGate(2 * math.asin(math.sqrt(0.75)), 0, 0, 0)

qubit_mapping = {
        0: 1,
        1: 0,
        2: 2,
        3: 3
    }

In [13]:
qc = QuantumCircuit(4)
qc.append(edge_gates["01"], [qubit_mapping[1], qubit_mapping[0]])
qc.append(edge_gates["12"], [qubit_mapping[1], qubit_mapping[2]])
# SWAP using noisy CNOTs
qc.append(edge_gates["03"], [qubit_mapping[1], qubit_mapping[3]])
qc.append(edge_gates["03"], [qubit_mapping[3], qubit_mapping[1]])
qc.append(edge_gates["03"], [qubit_mapping[1], qubit_mapping[3]])
# CNOT with newly swapped q3
qc.append(edge_gates["01"], [qubit_mapping[1], qubit_mapping[0]])
print(qc)

                                                            ┌───────────────┐»
q_0: ─────────■──────────────────■─────────────────■────────┤ U(2π/3,0,0,0) ├»
     ┌────────┴────────┐         │                 │        └───────┬───────┘»
q_1: ┤ U(2.9413,0,0,0) ├─────────┼─────────────────┼────────────────┼────────»
     └─────────────────┘┌────────┴────────┐        │                │        »
q_2: ───────────────────┤ U(2.4981,0,0,0) ├────────┼────────────────┼────────»
                        └─────────────────┘┌───────┴───────┐        │        »
q_3: ──────────────────────────────────────┤ U(2π/3,0,0,0) ├────────■────────»
                                           └───────────────┘                 »
«                                         
«q_0: ────────■─────────────────■─────────
«             │        ┌────────┴────────┐
«q_1: ────────┼────────┤ U(2.9413,0,0,0) ├
«             │        └─────────────────┘
«q_2: ────────┼───────────────────────────
«     ┌───────┴───────┐        

### Q3D. [20 Points] The figure below demonstrates a 5-qubit NISQ machine coupling map similar to the one we have seen in our lecture, where the value of each edge denotes the fidelity of a CNOT gate (in both directions) as defined in Q3. We would like to run the following logical quantum circuit on this 5-qubit NISQ machine.

<img src="./figures/q3d_device_topology.png" alt="Q3D Device" height=200/>
<img src="./figures/q3_logical_circuit.png" alt="Target Quantum Circuit" height=200/>

#### In `q3d_edge_gates` function in `question3.py`, create noisy CNOT gates defined in this question based on the device topology.
#### In `q3d_device_compatible_physical_circuit` function in `question3.py`, implement the mapped physical quantum circuit on the coupling map. You need to consider the optimal way to allocate the qubits to maximize the overall fidelity. You can only use 2-qubit noisy CNOT gates in `q3d_edge_gates`.
#### In `q3d_device_qubit_route_mapping` function in `question3.py`, provide your logical qubit to physical qubit mapping in a dictionary in the following format `{logical_qubit: physical_qubit}`, this is the dictionary you use to allocate logical qubits in the circuit to the physical qubits in the device.
#### In `q3d_device_qubit_read_mapping` function in `question3.py`, provide your physical qubit to logical qubit mapping in a dictionary in the following format `{physical_qubit: logical_qubit}`, this is the dictionary you use to read from the physical device post execution and retrive the result for the logical quantum circuit.